In [1]:
import os
import urllib.request

if not os.path.exists("the-verdict.txt"):
    url = "https://raw.githubusercontent.com/rasbt/LLMs-from-scratch/refs/heads/main/ch02/01_main-chapter-code/the-verdict.txt"
    file_path = "the-verdict.txt"
    urllib.request.urlretrieve(url, file_path)

In [2]:
with open("the-verdict.txt") as f:
    raw_text = f.read()

len(raw_text)

20479

In [11]:
import re

#Split the text into individual tokens using regular expressions
#Let's start of with a simple expression and text
# text = "Hello! This is a test."
# result = re.split(r'(\s)', text)
# print(result)

# result = re.split(r'[!.]|\s',text)
# result = [item for item in result if item.strip()]
# print(result)

#sophisticated regular expression with "the-verdict.txt"
new_text = "Hello, world. Is this -- a test?"
raw_result = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
raw_result = [item.strip() for item in raw_result if item.strip()]
processed = raw_result
len(processed)

4690

In [12]:
#Converting tokens into token IDs
all_words = sorted(set(processed))
vocab_size = len(all_words)
print(f"The size of the vocab is: {vocab_size}")

vocab = {token: integer for token, integer in enumerate(all_words)}

The size of the vocab is: 1130
